In [5]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

In [6]:
class AlexNet1D(nn.Module):
    def __init__(self, num_classes=10):
        super(AlexNet1D, self).__init__()
        self.features = nn.Sequential(
            # 第一层卷积层
            nn.Conv2d(1, 96, kernel_size=(11, 11), stride=(4, 4), padding=(0, 0)),  # 输入通道1，输出通道96
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=(3, 3), stride=(2, 2)),  # 最大池化层

            # 第二层卷积层
            nn.Conv2d(96, 256, kernel_size=(5, 5), padding=(2, 2)),  # 输入通道96，输出通道256
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=(3, 3), stride=(2, 2)),  # 最大池化层

            # 第三层卷积层
            nn.Conv2d(256, 384, kernel_size=(3, 3), padding=(1, 1)),  # 输入通道256，输出通道384
            nn.ReLU(inplace=True),

            # 第四层卷积层
            nn.Conv2d(384, 384, kernel_size=(3, 3), padding=(1, 1)),  # 输入通道384，输出通道384
            nn.ReLU(inplace=True),

            # 第五层卷积层
            nn.Conv2d(384, 256, kernel_size=(3, 3), padding=(1, 1)),  # 输入通道384，输出通道256
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=(3, 3), stride=(2, 2)),  # 最大池化层
        )
        self.avgpool = nn.AdaptiveAvgPool2d((6, 6))  # 自适应平均池化层
        self.classifier = nn.Sequential(
            nn.Dropout(),  # Dropout层
            nn.Linear(256 * 6 * 6, 4096),  # 全连接层
            nn.ReLU(inplace=True),
            nn.Dropout(),  # Dropout层
            nn.Linear(4096, 4096),  # 全连接层
            nn.ReLU(inplace=True),
            nn.Linear(4096, num_classes),  # 输出层
        )

    def forward(self, x):
        x = self.features(x)  # 卷积层
        x = self.avgpool(x)  # 池化层
        x = torch.flatten(x, 1)  # 展平
        x = self.classifier(x)  # 全连接层
        return x

In [7]:
# 检查是否有GPU可用
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 初始化模型
model = AlexNet1D(num_classes=2).to(device)

In [8]:
model

AlexNet1D(
  (features): Sequential(
    (0): Conv2d(1, 96, kernel_size=(11, 11), stride=(4, 4))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=(3, 3), stride=(2, 2), padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(96, 256, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=(3, 3), stride=(2, 2), padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(256, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): Conv2d(384, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): MaxPool2d(kernel_size=(3, 3), stride=(2, 2), padding=0, dilation=1, ceil_mode=False)
  )
  (avgpool): AdaptiveAvgPool2d(output_size=(6, 6))
  (classifier): Sequential(
    (0): Dropout(p=0.5, inplace=False)
    (1): Linear(in_features=9216, out_features=40